In [1]:
import ansys.aedt.core
import os
import tempfile
import time

In [2]:
from ansys.aedt.core import Maxwell3d


In [3]:
AEDT_VERSION = "2025.1"
NUM_CORES = 8
NG_MODE = False  # Open AEDT UI when it is launched.

In [4]:
"D:\KangDH\deVSimulation\Ansys"

'D:\\KangDH\\deVSimulation\\Ansys'

In [ ]:
# project_name = os.path.join("D:\EM_KDH\Moa_Edu_Sim\Basic", "PlanarCapacitor.aedt")

# 2D

In [ ]:
from ansys.aedt.core import Maxwell2d

m2d = Maxwell2d(
    project=project_name,
    version=AEDT_VERSION,
    new_desktop=False,
    non_graphical=NG_MODE,
)
rect1=m2d.modeler.create_rectangle([0, 0], [10, 10], name="rect1", matname="copper")
rect1.id
m2dModel=m2d.modeler
m2dModel.object_list   # List of objects in the model
m2dModel.object_names
rect1_1=m2dModel.get_object_from_name("rect1_1")

rect1=m2dModel.get_object_from_name("rect1")
rect1.color='Red'
Vertrect=rect1.vertices
rect1.fillet(Vertrect[1],None,0.5,0)


# 3D 

In [ ]:
m3dOdesign=m3d.odesign
m3dOdesign.SetSolutionType("Electrostatics")
m3dModel=m3d.modeler

## fillet 3D box edges with a given radius

In [ ]:
box1Obj=m3dModel.create_box([0,0,0],[10,10,10],name="box1",matname="copper")
box13D_vertices=box1Obj.vertices
box13D_vertices
box13D_edge=box1Obj.edges
box13D_vertices
edge4filltet=m3dModel.get_edgeids_from_vertexid(31,box1Obj.id)
box1Obj.fillet(None,edge4filltet[0:3],0.5,0)

## Electrostatics

In [ ]:
materials = m3d.materials
# conduc=materials.mat_names_aedt   

In [ ]:
downPlate=m3dModel.create_box([0,0,0],[25,25,2],name="downPlate",material="pec")
upPlate=m3dModel.create_box([0,0,3],[25,25,2],name="upPlate",material="pec")

In [ ]:
downPlate.color='Red'
upPlate.color='Red'

In [ ]:
Gap=m3dModel.create_box([0,0,2],[25,25,1],name="Gap",material="vacuum")

In [ ]:
Gap.color='Gray'

In [ ]:
downVoltage=m3d.assign_voltage(downPlate,0,"downVoltage")
upVoltage=m3d.assign_voltage(upPlate,0,"upVoltage")

In [ ]:
m3d.assign_matrix(["downVoltage","upVoltage"],'Matrix1')

In [ ]:
solSetupObj=m3d.create_setup("Setup1")
solSetupObj.available_properties
solSetupObj.props["MaximumPasses"]=10
solSetupObj.props["PercentError"]=1
solSetupObj.props["Enabled"]=1
solSetupObj.props["SolveMatrixAtLast"]="True"
solSetupObj.props["PercentRefinement"]=50
solSetupObj.props["MinimumPasses"]=2
solSetupObj.props["MinimumConvergedPasses"]=1


In [ ]:
m3d.valid_design
m3dAnalyObj=m3d.analyze()

## Cylindrical Capacitor


In [ ]:
m3dCylinder = Maxwell3d(
    project="D:\EM_KDH\Moa_Edu_Sim\Basic\CylinderCapacitor.aedt",
    version=AEDT_VERSION,
    new_desktop=False,
    non_graphical=NG_MODE,
)


In [ ]:
m3dCylyderOdesign=m3dCylinder.odesign
m3dCylyderOdesign.SetSolutionType("Electrostatic")

### geometry

In [ ]:
cylinderInnerObj=m3dCylinder.modeler.primitives.create_cylinder([0, 0, 0],
                         [0, 0, -4],
                         radius=0.6,
                         height=25,
                         num_sides=24,
                         name="Inner",
                         materials="copper")

In [ ]:
cylinderOuterObj=m3dCylinder.modeler.primitives.create_cylinder([0, 0, 0],
                         [0, 0, -4],
                         radius=1.2,
                         height=25,
                         num_sides=24,
                         name="Outer",
                         materials="copper")

In [ ]:
VaccumObj=m3dCylinder.modeler.primitives.create_cylinder([0, 0, 0],
                         [0, 0, -4],
                         radius=1,
                         height=25,
                         num_sides=24,
                         name="Gap",
                         materials="vaccum")

In [ ]:
m3dCylinderModel.subtract([cylinderOuterObj.id],[VaccumObj.id],False)

In [ ]:
RegionObj=m3dCylinderModel.create_region(
        pad_value=[300,300,300,300,0,0],
        pad_type='Percentage Offset',  
        name='Region',  
        material='Vacuum',
) 


### Assign

In [ ]:
m3dCylinder.assign_material(["Inner"],'copper')
m3dCylinder.assign_material(["Outer"],'copper')

m3dCylinderModel=m3dCylinder.modeler

In [ ]:
m3dCylinder.assign_voltage("Inner",0,"InnerVoltage")

In [ ]:
m3dCylyderOdesign.ChangeProperty(
	[
		"NAME:AllTabs",
		[
			"NAME:Maxwell3D",  # Tab Name
			[
				"NAME:PropServers", 
				"BoundarySetup:InnerVoltage"  # property_objectstr (전체다)
			],
			[
				"NAME:ChangedProps",
				[
					"NAME:Voltage",
					"Value:="		, "-1000V"
				]
			]
		]
	])
m3dCylinder.change_property(m3dCylyderOdesign,'Maxwell3D','BoundarySetup:InnerVoltage','Voltage','-1000V')

In [ ]:
m3dCylinder.assign_voltage("Outer",1000,"OuterVoltage")
m3dCylinder.assign_matrix(["InnerVoltage","OuterVoltage"],'Matrix1')
m3dCylinder.assign_force(["Inner"],coordinate_system='Global',is_virtual=True,force_name='Force1')

### Setup

In [ ]:
solCylinder3dSetupObj=m3dCylinder.create_setup("Setup1")
# solCylinder3dSetupObj.available_properties
solCylinder3dSetupObj.props["MaximumPasses"]=10
solCylinder3dSetupObj.props["PercentError"]=1
solCylinder3dSetupObj.props["Enabled"]=1
solCylinder3dSetupObj.props["SolveMatrixAtLast"]="True"
solCylinder3dSetupObj.props["PercentRefinement"]=50
solCylinder3dSetupObj.props["MinimumPasses"]=2
solCylinder3dSetupObj.props["MinimumConvergedPasses"]=1

### Post

In [ ]:
import ansys.aedt.core.visualization.post.common

In [ ]:
m3CylinderPost=m3dCylinder.post


In [ ]:
m3CylinderPost.plot_field('Mag_E','Region','Volume')

### def create_3d_plot

### def get_solution_data

### get Sol

In [ ]:
solCylinderPost.export_data_to_csv('D:\EM_KDH\Moa_Edu_Sim\Basic/test.csv')

In [ ]:
m3CylinderPost.export_field_plot(plot_name='Mag_E2',output_dir='D:\EM_KDH\Moa_Edu_Sim\Basic')

In [ ]:
m3dBattery.odesign.SetSolutionType("Electrostatic")

In [ ]:
a=m3CylinderPost.ofieldsreporter

In [ ]:
m3CylinderPost.plot_field_from_fieldplot(plot_name='E',project_path="",mesh_plot=True,image_format='png',view='isometric',plot_label='',show=True,)

## Parallel Plate Capacitor (Battery)

In [ ]:
m3dBattery=Maxwell3d(
    project="D:\EM_KDH\Moa_Edu_Sim\Basic\Battery.aedt",
    version=AEDT_VERSION,
    new_desktop=False,
    non_graphical=NG_MODE,
)

In [ ]:
m3dBatteryModel=m3dBattery.modeler

In [ ]:
helixBattery=m3dBatteryModel.create_udp(dll="RectHelix.dll",parameters=[""],library="syslib")

In [ ]:
RectHelixObj=m3dBatteryModel.get_object_from_name('RectHelix1:CreateUserDefinedPart:1')

In [ ]:
m3dBatteryModel.create_udp(dll="SegmentedHelix/RectHelix.dll",parameters=[["RectHeight","75mm"], ["RectWidth","2mm"],["StartHelixRadius","12mm"],["RadiusChange","6mm"],["Pitch","0mm"],["Turns","2.5"],["SegmentsPerTurn","0"],["RightHanded","1"]],library="syslib",name='RectHelixD')

In [ ]:
m3dBatteryModel.update_udp(assignment="RectHelixD",
                           )

In [ ]:
from ansys.aedt.core import generic

In [ ]:
m3d